# Image Reduction in Python 

Erik Tollerud (STScI)

In this notebook we will walk through several of the basic steps required to do data reduction using Python and Astropy.  This notebook is focused on "practical" (you decide if that is a code word for "lazy") application of the existing ecosystem of Python packages.  That is, it is *not* a thorough guide to the nitty-gritty of how all these stages are implemented.  For that, see other lectures in this session.

INSTALL CCDPROC (>=1.3), PHOTUTILS (>=0.4)

In [ ]:
import numpy as np

from astropy import units as u


%matplotlib inline
from matplotlib import pyplot as plt

## Getting the data

#  DOWNLOAD HERE

In [ ]:
ls -lh python_imred_data/

### Exercise 

Look at the ``observing_log`` file - it's an excerpt from the log.  Now look at the file sizes above.  What patterns do you see?  Can you tell why? (Hint: the ".gz" at the end is significant here.)

You might find it useful to take a quick look at some of the images with a fits viewer like `ds9` (or the visualizations below) if you aren't sure.

### Loading the Data

In [ ]:
from astropy.io import fits
from astropy import nddata

In [ ]:
data_g = fits.open('python_imred_data/ccd.037.0.fits.gz')
data_i = fits.open('python_imred_data/ccd.043.0.fits.gz')

### Quick look 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from astropy import visualization as aviz

In [ ]:
image = data_g[0].data
norm = aviz.ImageNormalize(image, interval=aviz.PercentileInterval(90), stretch=aviz.LogStretch())

fig, ax = plt.subplots(1,1, figsize=(6,10))
plt.colorbar(ax.imshow(image, norm=norm, origin='lower'))

In [ ]:
def show_image(image, percl=99, percu=None, figsize=(6, 10)):
    if percu is None:
        percu = percl
        percl = 100-percl
        
    norm = aviz.ImageNormalize(image, interval=aviz.AsymmetricPercentileInterval(percl, percu), 
                                      stretch=aviz.LogStretch())

    fig, ax = plt.subplots(1,1, figsize=figsize)
    plt.colorbar(ax.imshow(image, norm=norm, origin='lower'))
    
    return fig, ax

### Overscan and Bias 

In [ ]:
from glob import glob
import ccdproc

example of how it all works

In [ ]:
im1 = nddata.CCDData.read('python_imred_data/ccd.001.0.fits.gz', unit=u.count)
subed = ccdproc.subtract_overscan(im1, fits_section='[2049:2080,:]', overscan_axis=1)
trimmed = ccdproc.trim_image(subed, fits_section=subed.meta['DATASEC'])

In [ ]:
show_image(trimmed)

Note that it's note actually 4096

In [ ]:
arr = trimmed[:, 1024:].data.flatten()
plt.hist(arr, bins=100, histtype='step', log=True, range=(-100, 100))
np.mean(arr), np.std(arr)

In [ ]:
def overscan_correct(image):
    subed = ccdproc.subtract_overscan(image, fits_section='[2049:2080,:]', overscan_axis=1)
    trimmed = ccdproc.trim_image(subed, fits_section='[1:2048,1:4128]')
    return trimmed

#### Applied to data set 

In [ ]:
nd = nddata.CCDData.read('python_imred_data/ccd.037.0.fits.gz', unit=u.count)

In [ ]:
biasfns = glob('python_imred_data/ccd.00?.0.fits.gz') + ['python_imred_data/ccd.010.0.fits.gz']
biasfns

In [ ]:
biases = [overscan_correct(nddata.CCDData.read(fn, unit=u.count)) for fn in biasfns]

bias_combiner = ccdproc.Combiner(biases)
master_bias = bias_combiner.median_combine()

In [ ]:
show_image(master_bias)
master_bias

In [ ]:
arr = master_bias[:, 1024:].data.flatten()
plt.hist(arr, bins=100, histtype='step', log=True, range=(-50, 50))
np.mean(arr), np.std(arr)

In [ ]:
ccd_data_g = overscan_correct(nddata.CCDData.read(data_g.filename(), unit=u.count))

In [ ]:
ccd_data_g_unbiased = ccdproc.subtract_bias(ccd_data_g, master_bias)
show_image(ccd_data_g_unbiased, 10, 99.8)

### flat

In [ ]:
flat_g_fns = glob('python_imred_data/ccd.01[1-6].0.fits.gz')
flat_g_fns

In [ ]:
flats_g = [overscan_correct(nddata.CCDData.read(fn, unit=u.count)) for fn in flat_g_fns]
flats_g = [ccdproc.subtract_bias(flat, master_bias) for flat in flats_g]
flat_g_combiner = ccdproc.Combiner(flats_g)
combined_flat_g = flat_g_combiner.median_combine()

In [ ]:
show_image(combined_flat_g, 90)

In [ ]:
ccd_data_g_flattened = ccdproc.flat_correct(ccd_data_g_unbiased, combined_flat_g)
show_image(ccd_data_g_flattened, 10, 99.5)

Can't get rid of the amp glow... discuss/explain

## Photometry 

In [ ]:
import photutils

In [ ]:
plt.hist(ccd_data_g_flattened.data.flat, bins=100, range=(-100, 1000));

In [ ]:
from astropy.stats import SigmaClip

In [ ]:
bkg_estimator = photutils.MMMBackground(sigma_clip=SigmaClip(sigma=3.))
bkg_val = bkg_estimator.calc_background(ccd_data_g_flattened)
bkg_val

In [ ]:
ccd_data_g_bkgsub = ccd_data_g_flattened.subtract(bkg_val*u.count)

In [ ]:
plt.hist(ccd_data_g_bkgsub.data.flat, bins=100, range=(-100, 100));

In [ ]:
zoomed = ccd_data_g_bkgsub[2200:3300, :1000]
show_image(zoomed, 12, 99.9, figsize=(12, 10))

Why more extended than SDSS?

In [ ]:
scale_eq = u.pixel_scale(0.182*u.arcsec/u.pixel)
(25*u.pixel).to(u.arcsec, scale_eq)
(3*u.arcsec).to(u.pixel, scale_eq)

In [ ]:
positions = [(736., 401.5), (743., 672.)]
apertures = photutils.CircularAperture(positions, r=16.5/2)

show_image(zoomed, 12, 99.9, figsize=(6, 10))
apertures.plot(color='red')
plt.xlim(650, 800)
plt.ylim(350, 700)

SDSS photobj are: 20.04009 (S) and  20.4847 (N)

In [ ]:
20.4847 - 20.04009

In [ ]:
apphot = photutils.aperture_photometry(zoomed, apertures)
apphot

In [ ]:
mags = u.Magnitude(apphot['aperture_sum'])
mags[1] - mags[0]

Guess we're on the right track!

### Now find things automatically

In [ ]:
res = photutils.detect_sources(ccd_data_g_bkgsub.data, 40, 5)

In [ ]:
plt.figure(figsize=(8, 16))
plt.imshow(res, cmap=res.cmap('#222222'), origin='lower')